In [7]:
from dotenv import load_dotenv
load_dotenv()
import os 
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec


In [5]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="shaun",dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [6]:
import json 
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Jane Smith',
  'subject': 'Calculus',
  'stars': 4,
  'review': 'Dr. Smith is very knowledgeable and makes complex topics easier to understand.'},
 {'professor': 'Professor John Doe',
  'subject': 'Physics',
  'stars': 3,
  'review': 'Professor Doe has a deep understanding of the subject but sometimes goes too fast.'},
 {'professor': 'Dr. Emily Johnson',
  'subject': 'Biology',
  'stars': 5,
  'review': 'Dr. Johnson is enthusiastic and makes the classes very engaging and informative.'},
 {'professor': 'Dr. Michael Brown',
  'subject': 'Chemistry',
  'stars': 2,
  'review': "Dr. Brown's lectures are hard to follow, and the exams are very difficult."},
 {'professor': 'Professor Sarah Lee',
  'subject': 'Computer Science',
  'stars': 4,
  'review': 'Professor Lee is a great instructor with a lot of real-world experience.'},
 {'professor': 'Dr. Robert Wilson',
  'subject': 'History',
  'stars': 3,
  'review': "Dr. Wilson's lectures are informative, but the course materi

In [10]:
processed_data= []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }

    })

In [11]:
processed_data[0]

{'values': [-0.0075402227,
  -0.012658719,
  -0.047092848,
  0.0153621975,
  0.02522352,
  0.0045617004,
  0.012477593,
  0.024566099,
  0.016167203,
  -0.028389877,
  0.011028581,
  -0.002664905,
  -0.018743223,
  0.008372062,
  0.03721811,
  0.0023680592,
  -0.03933796,
  -0.0123903835,
  0.021681493,
  0.03228074,
  0.049749367,
  -0.019239642,
  0.034990925,
  -0.026189527,
  -0.03448109,
  -0.03743278,
  0.0269677,
  0.034212753,
  0.01875664,
  -0.0044510122,
  0.0677815,
  -0.010706579,
  -0.010324202,
  0.012960596,
  -0.03359558,
  0.0031998986,
  -0.027611705,
  0.007345679,
  -0.007292012,
  0.010096116,
  0.049373697,
  0.02750437,
  -0.059946112,
  -0.008465979,
  0.04414116,
  -0.029946221,
  -0.025974859,
  -0.026645698,
  0.012766053,
  0.045375504,
  -0.045187667,
  0.032414906,
  0.021144824,
  -0.034883592,
  -0.051278878,
  0.034964092,
  0.020514235,
  0.037083942,
  -0.0035118386,
  -0.03236124,
  0.061287787,
  0.0073054293,
  -0.013215515,
  -0.009693613,
  -0.0

In [12]:
index = pc.Index('shaun')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 21}

In [13]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 21}},
 'total_vector_count': 21}